# Gardening Household Count

### Importing the required packages

In [1]:
#Importing packages

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
from dotenv import load_dotenv
import plotly.express as px

### Downloading 'All gardens' and Household Count files from Commcare

In [2]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('COMCARE_EMAIL')
password_cc = os.getenv('COMCARE_PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#Creating login function
def commcare_all_gardens():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/789629a97bddd10b4648d5138d17908e/'
    )
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#All_gardens
commcare_all_gardens()

#Download the database "All gardens"
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()        

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def commcare_household_case():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/091321e0524e3ae7f5d5c1d3f43bd9c4/'
    )
    
commcare_household_case()

#Download the database "Household count"
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

def commcare_household_form():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/091321e0524e3ae7f5d5c1d3f43dccca/'
    )
    
commcare_household_form()


driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

time.sleep(5)

True

### Importing the datasets

In [3]:
all_gardens = pd.read_excel(f"~/Downloads/All Gardens {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
household = pd.read_excel(f"~/Downloads/houshold_garden {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases')
household_form = pd.read_excel(f"~/Downloads/Caris Jardinage - Jaden Lakou - Comptage Menage {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Forms')

In [4]:
household_cases = household[household['closed'] == False]
household_cases.head(2)

,number,caseid,name,age_in_year,caregiver_yes_no,dob,father_or_mother_has_dead_of_hiv,father_or_mother_has_hiv,first_name,gardining_member_relationship,...,indices.Garden,closed,closed_by_username,closed_date,last_modified_by_user_username,last_modified_date,opened_by_username,opened_date,owner_name,case_link
0,0,16d4e0eb-2ea0-45c6-ab8c-cb980b08e6b5,Amicius Roseberline,17,---,2003-12-26,3,3,Roseberline,2.0,...,93c5ad1d-4b58-4521-b1e9-222f95d2f5ae,False,---,---,j1james,2021-07-02 20:18:21,j1james,2021-07-02 20:18:21,j1james,https://www.commcarehq.org/a/caris-test/report...
1,1,a5ad9762-ba87-4dde-a293-98696fd167c6,Ciné Louisemana,62,1,1958-07-05,---,---,Louisemana,1.0,...,93c5ad1d-4b58-4521-b1e9-222f95d2f5ae,False,---,---,j1james,2021-07-02 20:18:21,j1james,2021-07-02 20:18:21,j1james,https://www.commcarehq.org/a/caris-test/report...


### Statistics

#### Number of household counted

In [5]:
all_gardens = all_gardens[(all_gardens.closed == False) & (all_gardens['beneficiary_type'].isin(['malnutrition_program', 'neighbor', 'dreams']))]
#---------------------------------------------------------------------------------------------------------------------------------------

household_form = household_form.drop_duplicates('form.case.@case_id').reset_index(drop = True)
household_form = household_form[household_form['form.case.@case_id'].isin(all_gardens['info.case_id'])]
household_form['username'].replace(['j3carmelle', 'j1louis'], ['3carl', '1mackenson'], inplace = True)
household_grouped = household_form.groupby('username').count()[['form.case.@case_id']]
household_grouped.reset_index(inplace = True)
household_grouped.rename(columns = {'form.case.@case_id': 'Nombre de ménage compté',
                                   'username': 'agent_code'}, inplace = True)
household_grouped.sort_values('Nombre de ménage compté', ascending = False, inplace = True)
household_grouped

,agent_code,Nombre de ménage compté
20,j1seide,553
25,j3plaisime,394
32,j6arison,390
43,j6yvenes,328
9,j1edner,316
7,j1appolon,312
23,j3donathan,303
37,j6jeune,290
26,j4beauvais,288
36,j6guerby,279


In [6]:
# household_form.to_excel('Test_household.xlsx', index = False)

In [7]:
# all_gardens = all_gardens[(all_gardens.closed == False) & (all_gardens['beneficiary_type'].isin(['Caris']))]
# #---------------------------------------------------------------------------------------------------------------------------------------

# household_form = household_form.drop_duplicates('form.case.@case_id').reset_index(drop = True)
# household_form = household_form[household_form['form.case.@case_id'].isin(all_gardens['info.case_id'])]
# household_grouped = household_form.groupby('username').count()[['form.case.@case_id']]
# household_grouped.reset_index(inplace = True)
# household_grouped.rename(columns = {'form.case.@case_id': 'Nombre de ménage compté',
#                                    'username': 'agent_code'}, inplace = True)
# household_grouped.sort_values('Nombre de ménage compté', ascending = False, inplace = True)
# household_grouped.head()

In [8]:
print(f"Number of household counted: {np.sum(household_grouped['Nombre de ménage compté'])}.")

Number of household counted: 9128.


#### Household counted by agent

In [9]:
px.bar(household_grouped, x = 'agent_code', y = 'Nombre de ménage compté')

In [11]:
lookup_agent = pd.read_excel('./lookup_gardening.xlsx', sheet_name='Sheet1')
lookup_agent.rename(columns = {'username': 'agent_code',
                              'name': 'Agent_name'}, inplace = True)
lookup_agent.head()

,agent_code,Agent_name,office
0,j1appolon,Appolon Willy,Cap-Haïtien
1,j1cepoudy,Cepoudy Elide,Cap-Haïtien
2,j1fadeline,Fadeline Antenor,Cap-Haïtien
3,j1james,James Jean Pierre,Cap-Haïtien
4,j1louisjeune,Louis Jeune Dadie,Cap-Haïtien


### Beneficiaries infos

In [12]:
agent_grouped_all = all_gardens.groupby('info.owner_name').count()[['info.case_id']]
agent_grouped_all.reset_index(inplace = True)
agent_grouped_all.rename(columns = {'info.case_id': 'Nombre de ménage à compter',
                                   'info.owner_name': 'agent_code'}, inplace = True)
agent_grouped_all

,agent_code,Nombre de ménage à compter
0,1jdensy,158
1,1mackenson,213
2,3carl,246
3,5loudelene,49
4,5mackendy,60
5,6jkenson,160
6,6wanes,110
7,j1appolon,311
8,j1cepoudy,162
9,j1edner,316


In [13]:
hh_td_depart = all_gardens.groupby('address_department').count()[['info.case_id']]
hh_td_depart.reset_index(inplace = True)
hh_td_depart.rename(columns = {'info.case_id': 'Ménage global'}, inplace = True)
hh_td_depart

,address_department,Ménage global
0,artibonite,2444
1,centre,2
2,grandanse,986
3,nippes,112
4,nord,2492
5,nord-est,1215
6,nord-ouest,842
7,ouest,87
8,sud,956


In [14]:
# agent_grouped_all.to_excel('Agent_grouped.xlsx', index = False)
# household_grouped.to_excel('Household_grouped.xlsx', index = False)

In [15]:
summary_table = agent_grouped_all.merge(household_grouped, on = 'agent_code', how = 'left')
summary_table

,agent_code,Nombre de ménage à compter,Nombre de ménage compté
0,1jdensy,158,158
1,1mackenson,213,213
2,3carl,246,241
3,5loudelene,49,49
4,5mackendy,60,58
5,6jkenson,160,160
6,6wanes,110,49
7,j1appolon,311,312
8,j1cepoudy,162,162
9,j1edner,316,316


## Performance

In [16]:
index = ['agent_code','Agent_name', 'Nombre de ménage à compter', 'Nombre de ménage compté', 'Proportion effectuée en %', 'Reste à compter']
#-------------------------------------------------------------------------------------------------------------------------

summary_table.fillna(0, inplace = True)
summary_table['Proportion effectuée en %'] = round((summary_table['Nombre de ménage compté']/summary_table['Nombre de ménage à compter'])*100, 0)
summary_table['Reste à compter'] = summary_table['Nombre de ménage à compter'] - summary_table['Nombre de ménage compté']
summary_table.sort_values(by='Proportion effectuée en %', ascending = False, inplace = True)
summary_table['Proportion effectuée en %'] = round((summary_table['Nombre de ménage compté']/summary_table['Nombre de ménage à compter'])*100, 0).astype(str) + '%'
summary_table = summary_table.reset_index(drop = True)
summary_table = summary_table.merge(lookup_agent[['agent_code', 'Agent_name']], on = 'agent_code', how = 'left')
summary_table = summary_table.reindex(columns = index)
summary_table = summary_table.drop(index = summary_table[summary_table['agent_code'] == 'pierrerobentz.cassion@carisfoundationintl.org'].index)
summary_table

,agent_code,Agent_name,Nombre de ménage à compter,Nombre de ménage compté,Proportion effectuée en %,Reste à compter
0,j6arison,Arison Edmond,329,390,119.0%,-61
1,1jdensy,Densy Sanon,158,158,100.0%,0
2,j3donathan,Donathan Hyppolite,303,303,100.0%,0
3,j3hergy,Hergy Janvier,129,129,100.0%,0
4,j3plaisime,Plaisime Natacha,394,394,100.0%,0
5,j4beauvais,Beauvais Clennecy,288,288,100.0%,0
6,j4frantz,Frantz Paul,263,263,100.0%,0
7,j4wilnes,Wilnes Joassaint,212,212,100.0%,0
8,j5hilaire,Hilaire Ginette,129,129,100.0%,0
9,j5oligene,Oligene François,159,159,100.0%,0


## Household to be counted

In [17]:
rest_hh = all_gardens[(~all_gardens['info.case_id'].isin(household_form['form.case.@case_id'])) & (all_gardens.closed == False) & (all_gardens['last_name'] != '---')][['info.owner_name', 'info.case_id', 'beneficiary_type', 'name', 'last_name', 'caris_site', 'info.last_modified_date', 'address_department']]
rest_hh.head(5)

,info.owner_name,info.case_id,beneficiary_type,name,last_name,caris_site,info.last_modified_date,address_department
3780,3carl,d9783ceb-d2c3-48fb-9eee-a8415edb90de,malnutrition_program,Nadege,Clerger,CAY/HICC - Hopital Immaculee Conception des Cayes,2022-02-17 14:09:18,sud
3781,3carl,87b60356-e579-4b7f-b823-9e3aa978750a,malnutrition_program,Micheline,Jean,CAY/HICC - Hopital Immaculee Conception des Cayes,2022-02-17 14:09:18,sud
3783,3carl,acdfcddc-3cd4-469c-9d4a-4755375ac04e,malnutrition_program,Fina,Pierre,CAY/HICC - Hopital Immaculee Conception des Cayes,2022-02-17 14:09:18,sud
4972,j1jnpierre,3ac3c5c1-3be0-4811-befe-c58327f944ee,malnutrition_program,awens,toussaint,MOR/CMSM - CAL de Mont Organise,2022-09-16 14:01:29,nord-est
10365,j4francois,b8510fec-e863-4bc9-ba8e-65c4c82ec44f,malnutrition_program,Jose,Maxceus,DMA/HCDM - Hopital de Communaute Dame-Marienne,2022-08-22 13:28:04,grandanse


## Data by department

In [18]:
hh_depart = rest_hh.groupby('address_department').count()[['info.case_id']]
hh_depart.reset_index(inplace = True)
hh_depart.rename(columns = {'info.case_id': 'Ménage à compter'}, inplace = True)
all_hh = hh_td_depart.merge(hh_depart, on = 'address_department', how = 'left')
all_hh = all_hh.fillna(0)
all_hh['Proportion à réaliser en %'] = round((all_hh['Ménage à compter']/all_hh['Ménage global'])*100, 0).astype(str) + '%'

all_hh

,address_department,Ménage global,Ménage à compter,Proportion à réaliser en %
0,artibonite,2444,3.0,0.0%
1,centre,2,0.0,0.0%
2,grandanse,986,2.0,0.0%
3,nippes,112,0.0,0.0%
4,nord,2492,0.0,0.0%
5,nord-est,1215,1.0,0.0%
6,nord-ouest,842,0.0,0.0%
7,ouest,87,0.0,0.0%
8,sud,956,5.0,1.0%


## Export data

In [19]:
import openpyxl

with pd.ExcelWriter('Household_Gardening_followup ' + str(datetime.today().strftime('%Y-%m-%d')) + '.xlsx', 
                        engine = 'openpyxl') as writer:
    summary_table.to_excel(writer, sheet_name = 'Performance_agents', index = False, na_rep = '---')
    rest_hh.to_excel(writer, sheet_name = 'Bénéficiaires à compter', index = False)
    all_hh.to_excel(writer, sheet_name = 'A compter par département', index = False)


In [20]:
driver.quit()